### Biblioteca - Riot API

**Equipe:** Camila Lopes e Giovani Moutinho  
**Disciplina:** Tópicos em Inteligência Computacional  
**Professora:** Luciana Balieiro Cosme  
**Curso:** Ciência da Computação   
  
**Referências:**  
[Documentação da API](https://developer.riotgames.com/api-methods/)

In [28]:
from lxml import html
import json
import requests
from decimal import Decimal
import urllib.request
from pprint import pprint

In [29]:
## Configurações
API_KEY = 'RGAPI-82b32172-5558-461f-91f5-18cc9d13fb10'
REGIAO = 'br1'
URL_BASE = 'https://'+REGIAO+'.api.riotgames.com/'
NICK_TESTE = 'TheDarkDescent'

In [87]:
class Summoner:
    nick = ""
    linkInfoPerfil = ""
    infoPerfil = ""
    accountId = ""
    profileIconId = ""
    summonerLevel = ""
    id = ""
    revisionDate = ""
    
    def __init__(self, nick):
        self.nick = nick
        self.linkInfoPerfil = URL_BASE+'lol/summoner/v3/summoners/by-name/'+self.nick+'?api_key='+API_KEY
        self.infoPerfil = requests.get(self.linkInfoPerfil)
        if self.infoPerfil.ok==True:
            self.infoPerfil =  self.infoPerfil.json()
            self.accountId = str(self.infoPerfil['accountId'])
            self.profileIconId = str(self.infoPerfil['profileIconId'])
            self.summonerLevel = str(self.infoPerfil['summonerLevel'])
            self.id = str(self.infoPerfil['id'])
            self.revisionDate = str(self.infoPerfil['revisionDate'])

In [98]:
class Champions:
    mainChampionId = ""
    mainChampionPoints = ""
    mainChampionName = ""
    playerId = ""
    
    def getChampionName(self, championId):
        campeoes = json.load(open('lol-champions.json', 'rb'))
        n = len(campeoes)-1
        while (n>-1):
            if campeoes[n]['key']==championId:
                return campeoes[n]['name']
            n=n-1
    
    def __init__(self, summoner):
        if(summoner.accountId != ''):
            self.playerId = summoner.id
            self.linkInfoChampions = URL_BASE+'lol/champion-mastery/v3/champion-masteries/by-summoner/'+self.playerId+'?api_key='+API_KEY
            self.infoChampions = requests.get(self.linkInfoChampions).json()
            self.mainChampionId = str(self.infoChampions[0]['championId'])
            self.mainChampionPoints = str(self.infoChampions[0]['championPoints'])
            self.mainChampionName = self.getChampionName(self.mainChampionId)

In [99]:
class MatchList:
    totalGames = ""
    matches = []
    totalHours = ""
    
    def getHorasJogadas(self):
        return str(int(int(self.totalGames)*35/60))
    
    def __init__(self, summoner):
        if(summoner.accountId != ''):
            self.linkInfoMatches = URL_BASE+'lol/match/v3/matchlists/by-account/'+summoner.accountId+'?beginIndex=201&api_key='+API_KEY
            self.infoMatches = requests.get(self.linkInfoMatches).json()
            self.totalGames = str(self.infoMatches['totalGames'])
            self.matches = self.infoMatches['matches']
            self.totalHours = self.getHorasJogadas()

In [100]:
class Leagues:
    allLeagues = []
    queueType = ""
    iHigher = -1
    tier = ""
    rank = ""
    wins = ""
    losses = ""
    leaguePoints = ""
    winrate = ""
    
    def getWinrate(self, wins, losses):
        return str(round(wins/(losses+wins)*100, 2))

    def listFilas(self):
        n = len(self.allLeagues)-1
        while (n>-1):
            print('Fila: '+ allLeagues[n]['queueType'])
            print('Elo: '+ allLeagues[n]['tier'])
            print('Divisão: '+ allLeagues[n]['rank'])
            print('Vitórias: '+ str(allLeagues[n]['wins']))
            print('Derrotas: '+ str(allLeagues[n]['losses']))
            print('Porcentagem de vitórias: ' + getWinrate(allLeagues[n]['wins'], allLeagues[n]['losses']) +'\n')
        
    def getRankNumber(self, tier, rank):
        ranks = json.load(open('lol-ranks.json', 'rb'))
        return int(ranks[tier][rank])
    
    def getHigherQueue(self):       
        n = len(self.allLeagues)-1
        maxRank = -1
        higherQueue = ""
        while (n>-1):
            if int(self.getRankNumber(self.allLeagues[n]['tier'], self.allLeagues[n]['rank'])) > int(maxRank):
                maxRank = int(self.getRankNumber(self.allLeagues[n]['tier'], self.allLeagues[n]['rank']))
                self.iHigher = n
                higherQueue = self.allLeagues[n]['queueType']
            n=n-1
        return higherQueue
    
    def getHigherInfo(self):
        self.tier = self.allLeagues[self.iHigher]['tier']
        self.rank = self.allLeagues[self.iHigher]['rank']
        self.wins = self.allLeagues[self.iHigher]['wins']
        self.losses = self.allLeagues[self.iHigher]['losses']
        self.leaguePoints = self.allLeagues[self.iHigher]['leaguePoints']
        self.winrate = self.getWinrate(self.allLeagues[self.iHigher]['wins'], self.allLeagues[self.iHigher]['losses'])
    
    def __init__(self, summoner):
        if(summoner.accountId != ''):
            self.linkInfoLeagues = URL_BASE+'lol/league/v3/positions/by-summoner/'+summoner.id+'?api_key='+API_KEY
            self.allLeagues = self.infoLeagues = requests.get(self.linkInfoLeagues).json()
            self.queueType = str(self.getHigherQueue())
            if self.iHigher != -1:
                self.getHigherInfo()

In [105]:
class Player:
    Summoner = ""
    Champions = ""
    MatchList = ""
    Leagues = ""
    
    def __init__(self, nick):
        self.Summoner = Summoner(nick)
        self.Champions = Champions(self.Summoner)
        self.MatchList = MatchList(self.Summoner)
        self.Leagues = Leagues(self.Summoner)